In [45]:
from IPython.core.display import display, HTML
import os
import chess
import chess.pgn

In [46]:
 boardColors = {
        'square dark': '#b0cae1',
        'square light': '#ffffff',
        'square light lastmove': '#f7ba48',
        'square dark lastmove': '#f7ba48',
        'margin': '#dedede',
        'coord': '#888888',
        'arrow green': '#05ff61',
        'arrow blue': '#00a2c7',
        'arrow red': '#c90028',
        'arrow yellow': '#c78800',
    }

In [50]:
class UtilsBoard:
    def __init__(self, pageTitle):
        
        path = '../shared/util_board-assets/' # path from notebooks
        
        assetsLayout = f'{path}basic-layout.html'
        html = open(assetsLayout)    
        self.html_layout = html.read().replace('$title',pageTitle)
        
        block11 = f'{path}block-1-1.html'
        html = open(block11)    
        self.html_block11 = html.read()
                
        block12 = f'{path}block-1-2.html'
        html = open(block12)    
        self.html_block12 = html.read()
        
        blocktext = f'{path}block-text.html'
        html = open(blocktext)    
        self.html_blocktext = html.read()
        
        block4x = f'{path}block-4-x.html'
        html = open(block4x)    
        self.html_block4x = html.read()
        
        cell = f'{path}cell.html'
        html = open(cell)    
        self.html_cell = html.read()
        
        row = f'{path}row.html'
        html = open(row)    
        self.html_row = html.read()
        
        self.html = ''
        
        self.rows = []

    def addRow(self, colBlock):
        contentRow = self.html_row.replace('$content', colBlock)
        self.rows.append(contentRow)
    
    def resetRows(self):
        self.rows = []
        
    def getCell(self, content, text='' ):
        return self.html_cell.replace('$content', content).replace('$text', text)

    
    def getBlockText(self, title, content ):
        return self.html_blocktext.replace('$title', title).replace('$content', content)
    
    def getBlock4x(self, content ):
        return self.html_block4x.replace('$content', content)
    
    def getBlock11(self, content1, content2):
        return self.html_block11.replace('$1', content1).replace('$2', content2)
    
    def getBlock12(self, content1, content2):
        return self.html_block12.replace('$1', content1).replace('$2', content2)
        
    def printHtml(self):
        contentHtml = self.html_layout.replace('$rows', ' '.join(self.rows) )
        self.html = contentHtml
        display(HTML(self.html))
    
    def mapArrows( self, listArrows ):
        '''
        Helper methode for svg arrow parameter.
        Returns a list of chess arrows with chess.squares (from,to) and color
        @listArrows: list of listArrow>[cellFrom:string, cellTo:string, colorLetter:string]

        Usage:
        arrows([['a3','d5','r']) >> [Arrow(A3, D5, color='red')]
        '''
        arrows = []
        if len(listArrows) == 0:
            return [] 
        for arrow in listArrows:
            squeareFrom = chess.parse_square(arrow[0])
            squeareTo = chess.parse_square(arrow[1])
            if arrow[2] == 'g':
                color = 'green'
            elif arrow[2] == 'y':
                color = 'yellow'
            elif arrow[2] == 'b':
                color = 'blue'
            else:
                color = 'red'
            arrows.append( chess.svg.Arrow(squeareFrom, squeareTo, color=color) )
        return arrows
    
    def mapCellsToSquare(self, listCells ):
        if len(listCells) == 0:
            return [] 
        listCellsMapped = [ chess.parse_square(cell) for cell in listCells ]
        return chess.SquareSet(listCellsMapped)
        
    def mapMove( self, listCells ):
        if len(listCells) == 0:
            return [] 
        squeareFrom = chess.parse_square(listCells[0])
        squeareTo = chess.parse_square(listCells[1])
        return chess.Move(squeareFrom, squeareTo)
    
    def uciToMove(self, uciCells):
        '''
        Convert a string uci notation "e2e4" to chess.Move()
        @uciCells : string
        '''
        squeareFrom= chess.parse_square(uciCells[0:1])
        squeareTo= chess.parse_square(uciCells[2:3])
        return chess.Move(squeareFrom, squeareTo)
    
    def uciToList(self, uciCells):
        '''
        Convert a string uci notation "e2e4" to string[]
        @uciCells : string
        '''
        return [ uciCells[0:2], uciCells[2:4]]
    
    def boardToSvg(self, board, _arrows=[], _lastmove=[], _squares=[], **kwargs ):
        
        _flipped = False
        if 'flipped' in kwargs:
            _flipped = kwargs.get("flipped")
        
            
        _check = None
        if 'check' in kwargs:
            _check = kwargs.get("check")
        
        boardsvg = chess.svg.board( 
            board=board, 
            size= 400,
            coordinates = True,
            flipped = _flipped,
            colors=boardColors,
            style= '',
            arrows= self.mapArrows(_arrows),
            lastmove = self.mapMove(_lastmove),
            squares = self.mapCellsToSquare(_squares),
            check = _check
        )
        return boardsvg
    
    def save(self, outputPath, fileNameWithExt):
        if not os.path.exists( outputPath ):
            os.makedirs( outputPath )
        with open( os.path.join(outputPath, fileNameWithExt) , "w") as file:
            file.write(self.html)
        
        
        

# Class Usage

In [35]:
# UtilsBoard = UtilsBoard()

In [36]:
'''
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish
import copy
from IPython.display import display


utilsBoard = UtilsBoard('Page Title')


fen = 'rnbqkb1r/pp3ppp/8/3np3/8/2NP1N2/PP3PPP/R1BQKB1R b KQkq - 0 1'
board  = chess.Board(fen)
boardSvg1 = utilsBoard.boardToSvg(board, _arrows=[['a3','d5','g']] )
block12_temp = utilsBoard.getBlock12('Example', boardSvg1)
utilsBoard.addRow(block12_temp)


boardSvg2 = utilsBoard.boardToSvg(board)
cell1 = utilsBoard.getCell(boardSvg2, 'Mira')
cells = ' '.join([cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1])
block4x_temp = utilsBoard.getBlock4x( cells )
utilsBoard.addRow(block4x_temp)


blocktext_temp = utilsBoard.getBlockText('Example', 'Lorem ipsum')
utilsBoard.addRow(blocktext_temp)

block11_temp = utilsBoard.getBlock11('Example', boardSvg1)
utilsBoard.addRow(block11_temp)


utilsBoard.printHtml()

utilsBoard.save(f'../../report/{_notebookFolder}','base.html')
'''

"\nimport chess\nimport chess.pgn\nimport chess.engine\nfrom stockfish import Stockfish\nimport copy\nfrom IPython.display import display\n\n\nutilsBoard = UtilsBoard('Page Title')\n\n\nfen = 'rnbqkb1r/pp3ppp/8/3np3/8/2NP1N2/PP3PPP/R1BQKB1R b KQkq - 0 1'\nboard  = chess.Board(fen)\nboardSvg1 = utilsBoard.boardToSvg(board)\nblock12_temp = utilsBoard.getBlock12('Example', boardSvg1)\nutilsBoard.addRow(block12_temp)\n\n\nboardSvg2 = utilsBoard.boardToSvg(board)\ncell1 = utilsBoard.getCell(boardSvg2, 'Mira')\ncells = ' '.join([cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1,cell1])\nblock4x_temp = utilsBoard.getBlock4x( cells )\nutilsBoard.addRow(block4x_temp)\n\n\nblocktext_temp = utilsBoard.getBlockText('Example', 'Lorem ipsum')\nutilsBoard.addRow(blocktext_temp)\n\nblock11_temp = utilsBoard.getBlock11('Example', boardSvg1)\nutilsBoard.addRow(block11_temp)\n\n\nutilsBoard.printHtml()\n\nutilsBoard.save('example.html')\n"

# Draw SVG arguments

### Last Movement: squares colored

In [20]:
lastmove = chess.Move(chess.G1, chess.F3),

### Check :  red mark in square

In [21]:
check  = chess.F3,

### X icon

In [22]:
squares = chess.SquareSet([chess.A8, chess.A1]),

### Arrows

In [24]:
### arrows
arrows=[
    chess.svg.Arrow(chess.D5, chess.D5, color="green"),
    chess.svg.Arrow(chess.D5, chess.C3, color="red"),
    chess.svg.Arrow(chess.D8, chess.A5, color="blue"),
    chess.svg.Arrow(chess.A5, chess.C3, color="yellow"),
],


#### Circle: same square
circle = chess.svg.Arrow(chess.D5, chess.D5, color="green"),

#### Red arrow
red = chess.svg.Arrow(chess.D5, chess.C3, color="red"),

#### Blue Arrow
blue = chess.svg.Arrow(chess.D8, chess.A5, color="blue"),

#### Yellow Arrow
yellow = chess.svg.Arrow(chess.A5, chess.C3, color="yellow"),


# Import in notebook

```
%run ../shared/util_board.ipynb
```